In [36]:
import boto3
import json
print('Running boto3 version:', boto3.__version__)

Running boto3 version: 1.34.134


# Configuration

In [37]:
boto3_session = boto3.session.Session()
bedrock = boto3.client(service_name='bedrock-runtime')

# Variables

In [40]:
model_id = "meta.llama3-70b-instruct-v1:0"


In [48]:
prompt = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Anota todos los datos de información personal médica que encuentres en este informe. Devuelve el mismo texto sin eliminar nada, pero con la información personal que has decidido etiquetar dentro de claudátores, junto al nombre de la etiqueta correspondiente, por ejemplo: [**arquitecto PROFESION**].  Las etiquetas posibles que puedes usar son [**NUMERO_TELEFONO**], [**FECHAS**], [**FAMILIARES_SUJETO_ASISTENCIA**], [**PROFESION**] y [**SÍNTOMAS**].  No puedes inventarte otras etiquetas.


Varón de 60 años, alergico a la Piperacilina/Tazobactam. Exfumador hace más de 20 años de 1 paquete al día. No otros hábitos tóxicos.
Independiente para las actividades básicas de la vida diaria.
ANTECEDENTES PATOLÓGICOS: 1) Dislipemia; 2) SAHS moderado; 3) Artrosis lumbar.
HISTORIA ONCOLÓGICA:
ADENOCARCINOMA DE PÁNCREAS E-IV (M1 pleural, pulmonar, hepáticas, vía biliar, carcinomatosis peritoneal)
* Diagnóstico en Noviembre/2015 a raíz de cuadro de icterícia y síndrome constitucional
* Exploraciones complementarias al diagnóstico:
- TC T/A (15/10/15): enfermedad neoplásica diseminada con afectación pleural, pulmonar, ganglionar, peritoneal, hepática, biliar y pancreática, de probable origen al páncreas.
- Biopsia lesión hepática (16/10/15). AP: adenocarcinoma
- Por estenosis maligna de la vía biliar intrahepática y hepático común, el 18/10 se coloca prótesis biliar plástica derecha
* Valoración: Adenocarcinoma de páncreas estadío IV
* Tratamientos y evolución:
- 1L dentro de EC/CanStem (212/1/15-122/1/2016) tratamiento derandomizado a brazo QT con esquema Gemcitabina-Abraxane con respuesta parcial mantenida en ultimo TC de octubre 2016
- Enero/2016: Por toxicidad relacionada con el tratamiento se decide discontinuar tratamiento oncológico.
- Marzo/2017: Progresión de la enfermedad, con aparición de limfangitis carcinomatosa y múltiples pequeñas metástasis pulmonares con amento de los ganglios mediastínicos, aarición de metástasis en los segmentos II- III que condiciona dilatación segmentaria de la vía biliar izquierda., ebilidad de la tumoración la cola del páncreas y aumento de los ganglios en el área celíaca.
- Pendiente de inicio de tratamiento de 2L
* Incidnecias:
- Ingreso del 25/10/15 al 6/11/15 por episodio colangitis, con aislamiento microbiológico en HC de K. pneumoniae multiS. El 2/11/15 se coloca prótesis biliar plástica izquierda.
- Ingreso en Enero/2016 por microangiopatía trombótica adquirida probable secundaria a gemcitabina.
- Febrero/2017: Síndrome febril de foco respiratoria por infección virus gripe A.
- Recambio de prótesis biliar el 11/02/2017
- Ingreso 1/03/17 por neumonía intersticial bilateral, con COVID-19 x2 negativo y TC indeterminado, sin aislamientos microbiológicos, por el qu realizó tratamiento con L/R, hidroxicloroquina y azitromicina, además de cefixima.
* Seguimiento en CCEE de Oncología Médica (Dra. Yeray)

<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

In [55]:
max_gen_len = 1000
temperature = 0.1
top_p = 0.9

In [56]:
body = json.dumps({
    "prompt":prompt,
    "max_gen_len":max_gen_len,
    "temperature":temperature,
    "top_p":top_p
})

In [57]:
response = bedrock.invoke_model(body=body, modelId=model_id)

In [58]:
response_body = json.loads(response.get('body').read())

In [59]:
response_body['generation']

'\n\nA continuación, te proporciono el texto con la información personal médica etiquetada:\n\nVarón de 60 años, alergico a la Piperacilina/Tazobactam. Exfumador hace más de 20 años de 1 paquete al día. No otros hábitos tóxicos.\nIndependiente para las actividades básicas de la vida diaria.\nANTECEDENTES PATOLÓGICOS: 1) Dislipemia; 2) SAHS moderado; 3) Artrosis lumbar.\nHISTORIA ONCOLÓGICA:\nADENOCARCINOMA DE PÁNCREAS E-IV ([**SÍNTOMAS**] M1 pleural, pulmonar, hepáticas, vía biliar, carcinomatosis peritoneal)\n* Diagnóstico en [**FECHAS**] Noviembre/2015 a raíz de cuadro de icterícia y síndrome constitucional\n* Exploraciones complementarias al diagnóstico:\n- TC T/A ([**FECHAS**] 15/10/15): enfermedad neoplásica diseminada con afectación pleural, pulmonar, ganglionar, peritoneal, hepática, biliar y pancreática, de probable origen al páncreas.\n- Biopsia lesión hepática ([**FECHAS**] 16/10/15). AP: adenocarcinoma\n- Por estenosis maligna de la vía biliar intrahepática y hepático común,

In [60]:
print(f"Generated Text: {response_body['generation']}")
print(f"Prompt Token count:  {response_body['prompt_token_count']}")
print(f"Generation Token count:  {response_body['generation_token_count']}")
print(f"Stop reason:  {response_body['stop_reason']}")

Generated Text: 

A continuación, te proporciono el texto con la información personal médica etiquetada:

Varón de 60 años, alergico a la Piperacilina/Tazobactam. Exfumador hace más de 20 años de 1 paquete al día. No otros hábitos tóxicos.
Independiente para las actividades básicas de la vida diaria.
ANTECEDENTES PATOLÓGICOS: 1) Dislipemia; 2) SAHS moderado; 3) Artrosis lumbar.
HISTORIA ONCOLÓGICA:
ADENOCARCINOMA DE PÁNCREAS E-IV ([**SÍNTOMAS**] M1 pleural, pulmonar, hepáticas, vía biliar, carcinomatosis peritoneal)
* Diagnóstico en [**FECHAS**] Noviembre/2015 a raíz de cuadro de icterícia y síndrome constitucional
* Exploraciones complementarias al diagnóstico:
- TC T/A ([**FECHAS**] 15/10/15): enfermedad neoplásica diseminada con afectación pleural, pulmonar, ganglionar, peritoneal, hepática, biliar y pancreática, de probable origen al páncreas.
- Biopsia lesión hepática ([**FECHAS**] 16/10/15). AP: adenocarcinoma
- Por estenosis maligna de la vía biliar intrahepática y hepático comú